In [ ]:
%matplotlib inline
import sys
sys.path.append("..") # Adds the module to path

# deeptrack.properties

This example introduces the module deeptrack.properties.

## What is a property?

Each feature (instance of the class `Feature`, see [features_example](features_example.ipynb)) can have several properties (instances of the class `Property`).
A propety has a value accessible through the field `current_value`, whose data type is not restricted. 
This value is updated through a sampling rule (method `.update()`), which is passed to the class constructor on initialization. 

## What is a sampling rule?

The sampling rule determines how the value of a property is updated upon calling `.update()`.
A sampling rule is defined when an instance of the class Property is created and can be of any type. 
When calling `.update()`, the value of the property is updated according to the first of the following that applies:
    
1.  If the sampling rule has a method `.sample()`, call `.sample()` and return the output.
2.  If the sampling rule is a dictionary, create an exact copy of the dictionary and recursively update each item in the copy using the same set of rules, and return the new dictionary. 
3.  If the sampling rule is a `list`, create an exact copy of the list and recursively update each element in the copy using the same set of rules, and return the new list.
4.  If the sampling rule is an iterable, return the next value. If the iterable is exhausted, the value of the property is not changed.
5.  If the sampling rule is callable, call it and return the result.
6.  If none of the above apply, return the sampling rule itself.

In [ ]:
import numpy as np
from deeptrack.properties import Property

## 1. Property with a constant value

The simplest example of a property is one that does not change during an update call.
This is commonly either a number or a tuple, but can be any data type that will be evaluated by case 6.
If you want to have a constant property with a value that would be evaluated by cases 1-5 (e.g., a list or a function), you can  wrap it as the output of a lambda function.

In [ ]:
# NUMBER

P = Property(1)
print("The current value of the property is", P.current_value)

P.update(force_update=True) # Numbers are not changed after a update() call
print("The current value of the property is", P.current_value)

Note that in this example we call `.update(force_update=True)` instead of the much more common `.update()`. This is necessary in this example because the properties are not linked to an instance of `Feature` and, therefore, are never resolved (which happens when a `Feature` is resolved and resets the properties so that it can be updated again).

In [ ]:
# TUPLE

P = Property((1, [2, 3], None))
print("The current value of the property is", P.current_value)

P.update(force_update=True) # Tuples are not changed after a update() call
print("The current value of the property is", P.current_value)

In [ ]:
# WRAPPED LIST

P = Property(lambda: [np.random.rand, 1, {}])
print("The current value of the property is", P.current_value)

P.update(force_update=True) # Tuples are not changed after a update() call
print("The current value of the property is", P.current_value)

## 2. Property with a discrete random value 

Discrete randomness can be achieved by a function (case 5).

In [ ]:
# FUNCTION

P = Property(lambda: np.random.randint(0, 10))
for _ in range(5): 
    P.update(force_update=True)
    print("The current value of the property is", P.current_value)

In [ ]:
# BINARY CHOICE

P = Property(lambda: 1 if np.random.rand() > 0.75 else 0)
for _ in range(5): 
    P.update(force_update=True)
    print("The current value of the property is", P.current_value)

## 3. Property with a continuous random value 

Continuous randomness is typically achieved by passing a function that returns a coninuous random value. This function should take no input, as noted in case 5. To use a function that needs arguments, wrap it in a function that calls it with the correct arguments.

In [ ]:
# FUNCTION WITH NO INPUT

P = Property(np.random.rand)
for _ in range(5):
    P.update(force_update=True)
    print("The current value of the property is", P.current_value)

In [ ]:
# WRAPPED FUNCTION

P = Property(lambda: np.random.normal(1, 5))
for _ in range(5):
    P.update(force_update=True)
    print("The current value of the property is", P.current_value)

## 4. Property with a deterministically changing value

Deterministically changing properties can be achieved using either an iterator (case 4) or a function (case 5). For the output of a function to change deterministically between calls, it should reference some variable outside its definition. Once an iterator has been exhausted, it will always return its last value.

In [ ]:
# ITERATOR

P = Property(iter([1, 2, 3, 4, 5]))
for _ in range(10):
    P.update(force_update=True)
    print("The current value of the property is", P.current_value)

In [ ]:
# FUNCTION

fibbonacci = [1, 1]
def fibbonacci_sequence():
    fibbonacci.append(fibbonacci[-2] + fibbonacci[-1])
    return fibbonacci

P = Property(fibbonacci_sequence)
for _ in range(10):
    P.update(force_update=True)
    print("The current value of the property is", P.current_value)

## 5. Property with dependent value

The value of a property can be dependent on the value on some other property. It does this by accepting some keyword argument corresponding to the name of the independent property. Instances of `Feature` will handle this automatically.

In [ ]:
random_number = Property(lambda: np.random.rand())

def get_dependent_number(random_number=None):
    return random_number + 1

dependent_number = Property(get_dependent_number)

for _ in range(5):
    dependent_number.update(random_number=random_number, force_update=True)
    print("The current value of the independent property is", random_number.current_value)
    print("The current value of the dependent property is", dependent_number.current_value, "\n")

## 6. What is a SequentialProperty?

## 7. What is a PropertyDict?

Another class contained in the module deeptrack.properties is `PropertyDict`. This is a dictionary of properties (keys: name of properties; values: properties) complemented by utility methods to manage collections of properties. These include:

* `.current_value_dict()`, which creates and returns a dictionary with the current value of all properties in the PropertyDict (keys: name of properties; values: current values of the properties).
* `.update()`, which calls the method `.update()` on all properties in the PropertyDict.
* `.sample()`, which calls the method `.sample()` on all properties in the PropertyDict, and creates and returns a dictionary from the output (keys: name of properties; values: sample outputs of the properties).


In [ ]:
from deeptrack.properties import PropertyDict, Property

property_dict = PropertyDict(
    foo=Property(1),
    bar=Property(np.random.rand)
)

for _ in range(5):
    property_dict.update(force_update=True)
    print("The current values of the properties in property_dict are", property_dict.current_value_dict())